In [67]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime

# Global Model

In [128]:
no_of_noaa_records = 7431587

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'noaa_nam_2'
pipeline = [
    {
        "$sample": {"size": 74315}
    }
]
cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [129]:
def format_date(t):
    t = str(t)
    date = datetime.strptime(t, '%Y%m%d%H')
    return f'{date.year}-{date.month}-{date.day} {str(date.hour).zfill(2)}:{str(date.minute).zfill(2)}:{str(date.second).zfill(2)}'


df = df_original
df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
df

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909b5dccef3c16a9f7d5e4,2010-1-14 12:00:00,0,G3900930,41.257586,-82.042522,102296.0,99069.0,253.225159,264.729492,...,4.676483,0.000,0.0,270.723434,20360.507202,24035.177785,5.953953,0.0,0.18736,0.0
1,60909bd2ba60520913262824,2010-1-15 12:00:00,3,G0800870,40.017721,-104.047304,102992.0,85858.0,1465.475159,270.553894,...,0.041497,0.000,0.0,271.623764,26067.372131,24035.608697,6.470860,0.0,0.00000,0.0
2,60909cdbf427f4b2b209d958,2010-1-18 00:00:00,0,G5600290,44.692840,-109.566195,101073.0,73462.0,2601.225159,261.439102,...,1.477570,0.000,0.0,271.571350,22160.520935,24234.948206,3.880981,0.0,1.27408,0.0
3,60909ae8e71b89653d2bde75,2010-1-13 06:00:00,6,G4100370,43.500737,-120.665510,100864.0,84813.0,1425.475159,274.918381,...,5.733231,4.250,0.0,274.628525,28167.625427,11035.114831,11.989980,100.0,0.00000,0.0
4,609099f30394db46bd12c006,2010-1-11 00:00:00,6,G0600070,39.408113,-121.597511,102178.0,101585.0,46.475159,280.707962,...,-2.079071,0.000,0.0,282.190552,20360.479736,24227.428675,19.429266,100.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74310,60909b82e7a50b8b8c8e62d2,2010-1-14 18:00:00,3,G2900810,40.244754,-93.816921,101905.0,98463.0,275.975159,273.235611,...,-2.752478,0.000,0.0,271.556381,26160.510254,37.607694,9.209002,100.0,0.10920,0.0
74311,60909cf1ef92c8286ad82459,2010-1-18 00:00:00,6,G2700710,47.883399,-94.019697,101581.0,96386.0,413.975159,262.300308,...,-1.697479,0.000,0.0,269.886429,30567.326355,24234.418267,4.614982,0.0,0.13280,0.0
74312,609099fe8ef9fc08588fcbbe,2010-1-11 06:00:00,0,G2700110,45.548602,-96.526505,102731.0,98336.0,337.225159,263.964081,...,-4.535324,0.000,0.0,264.617569,22160.514832,24234.747136,8.469556,100.0,0.13808,0.0
74313,609099baf93f3f2c0fafd78c,2010-1-10 12:00:00,6,G0400050,35.512395,-111.255848,103090.0,85958.0,1484.975159,278.918365,...,-0.220123,0.000,0.0,271.675156,20360.482788,24231.413460,6.015907,100.0,0.00000,0.0


In [131]:
noaa_features = [
    "mean_sea_level_pressure_pascal",
    "surface_pressure_surface_level_pascal",
    "orography_surface_level_meters",
    "temp_surface_level_kelvin",
    "2_metre_temp_kelvin",
    "2_metre_dewpoint_temp_kelvin",
    "relative_humidity_percent",
    "10_metre_u_wind_component_meters_per_second",
    "10_metre_v_wind_component_meters_per_second",
    "total_precipitation_kg_per_squared_meter",
    "water_convection_precipitation_kg_per_squared_meter",
    "soil_temperature_kelvin",
    "pressure_pascal",
    "visibility_meters",
    "precipitable_water_kg_per_squared_meter",
    "total_cloud_cover_percent",
    "snow_depth_meters",
    "ice_cover_binary"
]

unique_timestamps = df['year_month_day_hour'].unique()
df_map = {}

for selected_feature in noaa_features:
    df_s = df[['year_month_day_hour', selected_feature]]
    means = []
    for t in unique_timestamps:
        mean = df_s[df_s['year_month_day_hour'] == t][selected_feature].mean()
        means.append(mean)
    df_means = pd.DataFrame(list(zip(unique_timestamps, means)), columns=['ds', 'y'])
#     df_means.index.name = 'ds'
    df_map[selected_feature] = df_means
    
df_map['mean_sea_level_pressure_pascal'].head()

,ds,y
0,2010-1-14 12:00:00,102212.554850
1,2010-1-15 12:00:00,102427.085850
2,2010-1-18 00:00:00,101182.639580
3,2010-1-13 06:00:00,102174.151277
4,2010-1-11 00:00:00,102747.603226


In [116]:
pickle.dump(df_map, open('pickles/noaa/df_map.pkl', 'wb'))

In [117]:
# Start by loading pickled 'df_map'
df_map = pickle.load(open('pickles/noaa/df_map.pkl', 'rb'))

In [132]:
# select one feature
df0 = df_map['mean_sea_level_pressure_pascal']
df0.head()
df0.columns

Index(['ds', 'y'], dtype='object')

## Build Global Model

In [134]:
def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return fit_m, df_train_future, df_train_forecast

fit_m, df_train, df_train_forecast = predict(df0)

Initial log joint probability = -2.02015
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       340.026    0.00119618       6205.51           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       347.638   6.00432e-05       2010.85           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       349.891    0.00194563       14165.1           1           1      346   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       352.741   0.000294412       4239.46           1           1      453   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        353.66   4.30424e-05       1226.97           1           1      564   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [188]:
global_model = fit_m

In [138]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df0_cv = cross_validation(fit_m, initial='2 days', period='1 days', horizon='1 days')
df0_p = performance_metrics(df0_cv)
df0_p.head()

INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

INFO:prophet:n_changepoints greater than number of observations. Using 8.


Initial log joint probability = -2.02
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       99.5234     0.0133659         10367      0.4166      0.4166      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        109.86    0.00447523       34808.5      0.9696      0.9696      247   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       110.879   0.000324645       2990.69           1           1      359   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       112.108   0.000734205       2458.97           1           1      470   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       112.265   6.11675e-06       576.431           1           1      580   
    Iter      log prob        ||dx||      ||grad||       alpha     

INFO:prophet:n_changepoints greater than number of observations. Using 11.


...

Initial log joint probability = -2.02007
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       247.446     0.0299858       32558.6           1           1      138   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       251.443    0.00921149       20905.1           1           1      256   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       253.238    0.00027131       4269.97           1           1      371   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       254.584    0.00136367       5441.74           1           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        255.97   0.000254056       3333.44           1           1      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:n_changepoints greater than number of observations. Using 24.


Initial log joint probability = -2.02016
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       271.872   0.000514403       5725.71           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       273.314   0.000242394       7213.43      0.1967      0.8346      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       280.163   0.000136406       8901.12       1.354      0.1354      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       283.142   0.000156606       2157.45           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       283.561   0.000140964        3242.1           1           1      579   
    Iter      log prob        ||dx||      ||grad||       alpha  

Initial log joint probability = -2.02013
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       312.062    0.00604464       7972.53           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       321.024   0.000525731       2394.94      0.3284           1      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       328.209    7.3618e-05       1384.28       1.316      0.1316      355   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       331.488    3.1282e-05       899.346           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       332.364   0.000332871       1341.81      0.2982           1      584   
    Iter      log prob        ||dx||      ||grad||       alpha  

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,0 days 06:00:00,8551.897334,92.476469,83.105487,0.000817,0.000699,0.000817,0.000
1,0 days 12:00:00,36077.301491,189.940258,165.216841,0.001624,0.001437,0.001624,0.125
2,0 days 18:00:00,125281.745993,353.951615,293.815574,0.002887,0.002345,0.002889,0.250
3,1 days 00:00:00,299149.801304,546.945885,418.104264,0.004113,0.003543,0.004121,0.125


In [139]:
model = fit_m
print(f'seasonality_prior_scale: {model.seasonality_prior_scale}')
print(f'changepoint_prior_scale: {model.changepoint_prior_scale}')
print(f'uncertainty_samples: {model.uncertainty_samples}')
print(f'seasonlity_mode: {model.seasonality_mode}')
print(f'interval_width: {model.interval_width}')
print(f'growth: {model.growth}')

seasonality_prior_scale: 10.0
changepoint_prior_scale: 0.05
uncertainty_samples: 1000
seasonlity_mode: additive
interval_width: 0.8
growth: linear


## GridSearch on Global Model

In [149]:
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')    


def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '2 days', period='1 days', horizon='1 days', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale  

In [147]:
rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df0)

# Optimal values
print(f'rmse: {rmse}')
print(f'changepoint_prior_scale: {changepoint_prior_scale}')
print(f'seasonality_prior_scale: {seasonality_prior_scale}')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = 137.119
Iteration  1. Log joint probability =    125.411. Improved by -11.7085.
Iteration  2. Log joint probability =    161.704. Improved by 36.2934.
Iteration  3. Log joint probability =    182.823. Improved by 21.1191.
Iteration  4. Log joint probability =    215.766. Improved by 32.943.
Iteration  5. Log joint probability =    242.688. Improved by 26.9216.
Iteration  6. Log joint probability =    247.787. Improved by 5.09965.
Iteration  7. Log joint probability =    250.059. Improved by 2.27196.
Iteration  8. Log joint probability =    251.532. Improved by 1.47284.
Iteration  9. Log joint probability =    251.621. Improved by 0.0890902.
Iteration 10. Log joint probability =    251.966. Improved by 0.345213.
Iteration 11. Log joint probability =    252.277. Improved by 0.310613.
Iteration 12. Log joint probability =    252.374. Improved by 0.0974569.
Iteration 13. Log joint probability =    252.489. Improved by 0.115004.
Iteration 14. Log joint proba

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = 118.698
Iteration  1. Log joint probability =    125.411. Improved by 6.71279.
Iteration  2. Log joint probability =    161.564. Improved by 36.1531.
Iteration  3. Log joint probability =    180.417. Improved by 18.8523.
Iteration  4. Log joint probability =    217.254. Improved by 36.8375.
Iteration  5. Log joint probability =    241.509. Improved by 24.2546.
Iteration  6. Log joint probability =    250.741. Improved by 9.23254.
Iteration  7. Log joint probability =    250.966. Improved by 0.225089.
Iteration  8. Log joint probability =    251.001. Improved by 0.0346655.
Iteration  9. Log joint probability =    251.578. Improved by 0.576878.
Iteration 10. Log joint probability =    252.357. Improved by 0.779213.
Iteration 11. Log joint probability =    252.471. Improved by 0.11369.
Iteration 12. Log joint probability =    252.651. Improved by 0.180092.
Iteration 13. Log joint probability =    252.782. Improved by 0.131051.
Iteration 14. Log joint proba

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00


...


Initial log joint probability = -55.0875
Iteration  1. Log joint probability =    125.344. Improved by 180.431.
Iteration  2. Log joint probability =    155.052. Improved by 29.7089.
Iteration  3. Log joint probability =    201.434. Improved by 46.3812.
Iteration  4. Log joint probability =    228.506. Improved by 27.0724.
Iteration  5. Log joint probability =    254.772. Improved by 26.2662.
Iteration  6. Log joint probability =    278.104. Improved by 23.3317.
Iteration  7. Log joint probability =     300.51. Improved by 22.4062.
Iteration  8. Log joint probability =    325.575. Improved by 25.0646.
Iteration  9. Log joint probability =    342.052. Improved by 16.4775.
Iteration 10. Log joint probability =     369.64. Improved by 27.588.
Iteration 11. Log joint probability =    371.412. Improved by 1.77216.
Iteration 12. Log joint probability =     372.05. Improved by 0.637813.
Iteration 13. Log joint probability =    372.322. Improved by 0.271881.
Iteration 14. Log joint probabilit

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = -73.5082
Iteration  1. Log joint probability =    125.342. Improved by 198.85.
Iteration  2. Log joint probability =    157.593. Improved by 32.2513.
Iteration  3. Log joint probability =    199.479. Improved by 41.8853.
Iteration  4. Log joint probability =    226.623. Improved by 27.1447.
Iteration  5. Log joint probability =    253.265. Improved by 26.642.
Iteration  6. Log joint probability =    290.156. Improved by 36.8902.
Iteration  7. Log joint probability =    296.059. Improved by 5.90313.
Iteration  8. Log joint probability =    347.689. Improved by 51.6306.
Iteration  9. Log joint probability =    368.353. Improved by 20.6641.
Iteration 10. Log joint probability =    372.261. Improved by 3.90716.
Iteration 11. Log joint probability =    372.339. Improved by 0.0789348.
Iteration 12. Log joint probability =     372.34. Improved by 0.000323827.
Iteration 13. Log joint probability =     372.34. Improved by 8.42056e-05.
Iteration 14. Log joint pro

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


Time: 706.3904054430313
rmse: 331.1993775284078
changepoint_prior_scale: 0.5
seasonality_prior_scale: 10.0


In [159]:
gis_joins = df_original['gis_join'].unique()
print(f'#GISJOINs: {len(gis_joins)}')

#GISJOINs: 3049


## Dictionary of GISJOIN --> DF

In [171]:
import dask
selected_feature = 'mean_sea_level_pressure_pascal'
child_dfs = {}

def get_df_by_gis_join(gis_join):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"gis_join": gis_join}}])
    df = pd.DataFrame(list(cursor))[['year_month_day_hour', selected_feature]]
    df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
    df.columns = ['ds','y']
    return df

futures = []
for gis_join in gis_joins:
    child_dfs[gis_join] = get_df_by_gis_join(gis_join)

G3900930 G0800870 G5600290 G4100370 G0600070 G1600150 G0801090 G4600930 G0800010 G3000710 G3700410 G1900750 G3800310 G4500070 G3900050 G5100710 G2200350 G0600690 G4601170 G3100210 G4900450 G5500290 G1200670 G4802090 G4800710 G3500060 G0100890 G1700210 G1301850 G5400870 G2500230 G2601390 G3700110 G0501190 G0101270 G0600790 G2600730 G1600430 G1701470 G0101130 G0600270 G4001130 G3800470 G2600330 G0500150 G4200890 G1700370 G1700470 G0500930 G5600070 G2200470 G2601510 G1600490 G3200070 G2700110 G3101150 G0800370 G5400750 G4200150 G5600050 G1600210 G0600990 G1200970 G0600890 G1201230 G3101110 G3200170 G5500710 G0400030 G4200310 G3600090 G1201270 G4200350 G1901870 G4700350 G0400230 G2000710 G2600070 G4802290 G3500390 G0600190 G0600710 G3900350 G5000210 G0800750 G5300210 G5600350 G1600350 G0600450 G5500050 G4600090 G1800870 G0201980 G2001870 G4601210 G1302250 G2700410 G3500590 G3500530 G4804650 G4804430 G0800250 G1900390 G2801590 G4200330 G2001490 G1300710 G2701430 G3500190 G4100250 G5400830 G

G5300330 G4900490 G5401070 G4601090 G5500370 G3900290 G4100490 G4804050 G3101250 G2700130 G0100110 G1901490 G3701130 G0600670 G3701030 G5500110 G0801250 G2200150 G2800490 G3100410 G2100010 G3001110 G2900490 G2200650 G5400510 G2001650 G2001110 G2001290 G0100770 G2001670 G0101070 G3400070 G0600250 G5500070 G0100530 G4701150 G3000230 G2200930 G2000690 G1700670 G3100450 G3000090 G2902030 G4601350 G0800490 G2801010 G2901030 G4803670 G2601450 G4804330 G2200970 G4802250 G3701370 G4400090 G2001450 G2601210 G5101570 G1300690 G1701510 G2600170 G1300530 G0100550 G1900810 G1300950 G4200850 G3600410 G1301410 G3000910 G3901250 G3700510 G5101870 G2200750 G2800350 G4000330 G4701770 G3000830 G5400930 G5500930 G4900530 G4701030 G4803650 G0801130 G4802510 G5600410 G2901230 G5400190 G2900810 G4100010 G0900110 G1801130 G2902070 G2800670 G5300610 G3100190 G3701750 G2901010 G2200010 G5500490 G1300330 G3000670 G5500130 G2601270 G0600230 G2901860 G3900090 G5300090 G1702030 G2701290 G3600830 G1301110 G1700010 G

G5400170 G1200010 G4801550 G2000670 G0600410 G1200830 G2102090 G3701290 G1600810 G5100630 G2500170 G1900570 G5100610 G2201150 G0100010 G3800370 G4001210 G2001130 G3101710 G2601190 G2701030 G1700310 G1901350 G3900870 G0600150 G2001230 G4700770 G3000510 G3400050 G1900930 G0501350 G1901710 G2101090 G2101410 G1900630 G4803730 G4804090 G3600350 G2000170 G3701830 G5100330 G4803610 G1701770 G5400050 G2900990 G1701890 G4600890 G0101090 G4803510 G1200410 G4803630 G4600630 G3701670 G4700450 G1900970 G5501330 G4600730 G2300070 G4802810 G1300350 G4804410 G1800570 G2002050 G4001350 G2600490 G3800510 G2101050 G3700830 G3701550 G2102070 G5100750 G2902210 G1901570 G5500830 G3100390 G2601630 G3600130 G1901170 G2000030 G2800990 G4701710 G2200290 G5000070 G4700310 G4200490 G5500390 G0500710 G5500870 G3701350 G3800810 G2600510 G0800030 G4001370 G0101230 G4500530 G4804450 G4001030 G4701690 G3901530 G1700890 G4001290 G4802110 G1200590 G1700610 G1303070 G2001210 G2000530 G4801190 G0100090 G2500130 G3500490 G

G2101590 G1600090 G4701390 G1700510 G4601270 G2100050 G3400090 G1800670 G4200930 G5400790 G0201000 G3901490 G4700230 G3701050 G5101990 G1000030 G4000110 G2901870 G2900770 G1600750 G2000010 G4804190 G2600290 G3900750 G4701010 G3900610 G4804710 G2001750 G4200730 G3901330 G2101130 G1901150 G4700510 G1800450 G2901730 G1300230 G4802630 G1300130 G2900950 G4700270 G4801490 G1701210 G1801650 G3701070 G4500210 G2101610 G0501130 G1301710 G3700550 G2000790 G5000250 G2100470 G1303050 G1301170 G2101030 G1302630 G2102370 G2400150 G2102230 G1300570 G4200950 G2101750 G2601650 G1900230 G3600230 G1700710 G1800730 G1300730 G3100430 G3300170 G4200370 G2100990 G2901770 G1300250 G1900470 G5400290 G5101550 G2900010 G1801790 G3901010 G4500050 G2200550 G0900130 G2601550 G1700330 G2102170 G4801230 G1301330 G2801410 G2100910 G3701990 G5100970 G2102130 G5100850 G3200290 G0100610 G1302930 G2200890 G2000950 G2300230 G2700090 G1701390 G4800070 G3900330 G1800650 G4800850 G5400430 G3901070 G3900150 G5101270 G3700190 G

In [173]:
pickle.dump(child_dfs, open('pickles/noaa/child_dfs.pkl', 'wb'))

In [197]:
class TLModel:    
    def __init__(self, gis_join, time, model):
        self.rmse = rmse
        self.gis_join = gis_join
        self.time = time
        self.model = model
        
    def __str__(self):
        return f'{self.gis_join}: TLModel(time={self.time})'
    
class NonTLModel:
    def __init__(self, gis_join, time):
        self.gis_join = gis_join
        self.time = time
        
    def __str__(self):
        return f'{self.gis_join}: NonTLModel(time={self.time})'

# Child Models (Non-TL)

In [198]:
import time

non_tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    predict(child_dfs[gis_join])
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    non_tl_models[gis_join] = NonTLModel(gis_join, time2)
    counter += 1

Initial log joint probability = -2.03418
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7549.01   0.000559323       45988.9           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7973.35    0.00255003       23047.9           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       8009.92   0.000187361       13642.2           1           1      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       8107.64    0.00118167       9648.77           1           1      466   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       8199.37   0.000231663       7333.39           1           1      574   
    Iter      log prob        ||dx||      ||grad||       alpha  

2 - G0800870: 11.20615096599795 seconds
Initial log joint probability = -2.79845
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       92707.1   0.000138645       31596.8           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199         93030      0.003145       76425.3           1           1      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       93117.3    0.00308266       82431.4      0.9893      0.9893      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399         93166   0.000322608       8401.13       0.811       0.811      462   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       93180.2    0.00225613        167697           1           1      568   
    Iter      log prob  

4 - G4100370: 62.94742774008773 seconds
Initial log joint probability = -2.21134
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       22074.7     0.0031645        285338           1           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199         22390   0.000460123       44521.1      0.3334           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       22748.9   0.000706037       63014.7      0.4618      0.4618      349   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       22802.7    0.00376281        100604           1           1      462   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       22904.4    0.00145136       53754.2           1           1      579   
    Iter      log prob  

6 - G1600150: 14.792307045077905 seconds
Initial log joint probability = -2.44618
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       39575.1   5.87214e-05         15575      0.2178      0.2178      135   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199         39657   0.000608439       14335.5           1           1      246   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       39826.5   0.000153429       5832.09       0.835       0.835      361   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       39872.1     0.0068814       32006.3      0.4769      0.4769      478   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       39887.1    0.00160325       22396.1           1           1      596   
    Iter      log prob 

9 - G0800010: 7.675041910028085 seconds
Initial log joint probability = -2.25554
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99         80751   0.000398847        137115      0.3159           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199         81979   0.000678333       64750.8           1           1      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       82330.9   3.23784e-05        140255      0.4419      0.4419      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       82790.9   0.000478632        236185           1           1      463   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       83156.2     0.0154675   1.28148e+06           1           1      571   
    Iter      log prob  

...

90 - G5500050: 8.648116844939068 seconds
Initial log joint probability = -2.04535
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       9193.08    0.00312758       64377.8           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       9525.71   0.000334044       22546.8      0.5661      0.5661      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       9577.82    0.00184554       18057.4           1           1      355   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       9597.56   0.000415708       19328.4      0.3131           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       9615.32   0.000639433       10569.8           1           1      580   
    Iter      log prob 

92 - G1800870: 5.306268791900948 seconds
Initial log joint probability = -2.32662
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       61763.8    0.00170308       70101.1      0.8989      0.8989      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       63339.7    0.00180161        272364           1           1      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       63996.8   4.23276e-05       30315.1           1           1      366   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       64659.9   6.38602e-05       23381.8           1           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       65730.1    0.00341245       58939.6           1           1      600   
    Iter      log prob 

95 - G4601210: 7.394172974862158 seconds
Initial log joint probability = -2.02169
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       748.374    0.00215086       8620.89           1           1      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        777.24    0.00621158       2901.85           1           1      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       783.324   0.000204205       949.696           1           1      355   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       786.898    0.00119801        1828.1      0.8563      0.8563      470   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       795.719   0.000337816       357.099      0.7384      0.7384      578   
    Iter      log prob 

97 - G2700410: 5.645052945008501 seconds
Initial log joint probability = -2.19849
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       49480.7   0.000896882       63263.1           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       49600.8   7.18349e-05       22936.2      0.2471      0.2471      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       49848.6    0.00307646         71471           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399         50012    0.00013809       26362.6      0.1529           1      472   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       50075.4    0.00238497       57371.4           1           1      586   
    Iter      log prob 

100 - G4804650: 12.733139841118827 seconds


# Child Models (TL)

In [199]:
def predict_transfer(df_train, trained_model):
    m_parent = trained_model
    # initilaize model with hyperparameters from parent model
    m = Prophet(
        seasonality_prior_scale = m_parent.seasonality_prior_scale,
        changepoint_prior_scale = m_parent.changepoint_prior_scale,
    )
    m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return m, df_train_future, df_train_forecast

tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    m, df_train_future, df_train_forecast = predict_transfer(child_dfs[gis_join], global_model)
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    tl_models[gis_join] = TLModel(gis_join, time2, m)
    counter += 1 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


Initial log joint probability = -2.03418
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7458.75     0.0366052       45531.3           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7668.24    0.00394961       17691.6           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       7849.71   0.000658998       32188.6           1           1      347   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       7990.92    0.00119234       9360.52           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       8025.72   9.10719e-05        4572.4      0.1594      0.1594      579   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


1 - G3900930: 4.039233192102984 seconds
Initial log joint probability = -2.13583
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       18798.8    0.00472404       76148.8           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       18962.5   0.000507454       33867.8           1           1      231   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       19044.1   0.000244076       13097.7           1           1      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       19126.4   0.000188566       7129.75           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       19165.7    0.00204759       16027.4           1           1      573   
    Iter      log prob  

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


...

97 - G2700410: 3.3173177039716393 seconds
Initial log joint probability = -2.19849
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       48847.4      0.021878        266575           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       49702.3    0.00485885        110114           1           1      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       49767.1   0.000671599       91335.9       0.476           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       49850.5   0.000194928         26235      0.4778      0.4778      471   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       49885.6   0.000608695       26212.2           1           1      582   
    Iter      log prob

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


98 - G3500590: 11.914384474046528 seconds
Initial log joint probability = -2.47046
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       86359.8     0.0372871        458900           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       87284.3    0.00335685        113374           1           1      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       87641.5    0.00196945        232072           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       87970.5    0.00225947        169878           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       88097.8   0.000365108       17949.1           1           1      576   
    Iter      log prob

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


99 - G3500530: 24.94009071099572 seconds
Initial log joint probability = -2.11695
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       44051.3    0.00186064        157473           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       44756.8   0.000237376       30790.7      0.8289      0.8289      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       44988.2   0.000429668         18067           1           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       45641.9   9.95543e-05       29601.9           1           1      465   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       45925.5   0.000991869        114135           1           1      583   
    Iter      log prob 

## Collect Times taken to build Models

In [214]:
import pandas as pd

non_tl_times = [x.time for x in non_tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(non_tl_models.keys(), non_tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('non_tl_times.csv', index=False)

In [215]:
tl_times = [x.time for x in tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(tl_models.keys(), tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('tl_times.csv', index=False)